In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def avg_stepsize_corr_b1(positions, nominal_positions, 
        outliers=0, outlier_removal_cutoff_sigma_ratio=1.0, sigma_multiplyer=3, od_ls=False, plot=False):

    nominal_step_size = nominal_positions[3] - nominal_positions[0]
    diffs =  np.diff( positions )

    if plot:
        plt.plot(range(len(diffs)), diffs, "o-b")

    if outliers == 0:
        avg_step_size = np.mean( diffs )
        od_std_b1     = np.std(diffs, ddof=1)
    else:
        excluded_indices = []

        diffs1  = np.array(diffs)
        for i in range(outliers):
            avg     = np.mean( diffs1 )
            i_extremum = np.argmax(np.abs(diffs1-avg))
            diffs1_ = diffs1[ np.arange(len(diffs1)) != i_extremum ]
            if np.abs(diffs1[i_extremum] - avg)<0.01:
                break
            elif np.std(diffs1_, ddof=1)/np.std(diffs1, ddof=1) > outlier_removal_cutoff_sigma_ratio:
                break
            excluded_indices.append( np.where(np.isclose(diffs, diffs1[i_extremum]))[0][0] )
            diffs1 = diffs1_
        
        avg = np.mean(diffs1)
        std = np.std(diffs1, ddof=1)
        od_std_b1 = std
        lim = sigma_multiplyer*std
        diffs2 = diffs[ np.abs( diffs-avg )<=lim ]
        avg_step_size = np.mean(diffs2)

        if plot:
            plt.plot(excluded_indices, diffs[np.array(excluded_indices)], "xr", markersize=12)
            plt.plot([-1,5], [avg, avg], "-", color="red")
            plt.fill_between([-1,5], [avg-lim, avg-lim], [avg+lim, avg+lim], color="red", alpha=0.2)
            plt.plot(np.arange(len(diffs))[ np.abs( diffs-avg )>lim ]+0.3, diffs[ np.abs( diffs-avg )>lim ], "*m", markersize=12)
    
    if plot:
        plt.plot([-1,5], [avg_step_size, avg_step_size], "--g")
        plt.show()

            
    od_estimates = diffs - avg_step_size
    if od_ls:
        od_estimates *= nominal_step_size/avg_step_size
    new_stepsizes = od_estimates + nominal_step_size
    new_positions = np.concatenate(([0], np.cumsum(new_stepsizes)))
    # take the middle point as reference
    new_positions = new_positions - new_positions[2] + nominal_positions[7]

    return new_positions, od_std_b1, avg_step_size/nominal_step_size

In [ ]:
params = {
    "Nominal_LS_B1":0.9975,
    "Nominal_LS_B2":0.995,
    "Doros_LS_B1":1.0025,
    "Doros_LS_B2":1.005,
    "Arc_LS_B1":1.005,
    "Arc_LS_B2":1.01,
    "Sigma_B1":90, #micron
    "Sigma_B2":90, #micron
    "nominal_step_size_B1": 130, #micron
    "nominal_step_size_B2": 130, #micron
    "Doros_noise_sigma":0.25, # micron, statistical fluctuation on top of true value
    "Arc_noise_sigma":0, # micron
    "rate_runcertainty_at_head_on":0.005, #relative
    "randomize_rates":True,
    "beamspot_position_uncertainty":0.25, #micron, looks like 0.15 um in Mahmoud's fits btw
    "randomize_beamspot_position":True,
}

In [ ]:
# parameters
sigma_od = 2
kick     = 8 

# code
od_b1 = np.random.normal(size=15)*sigma_od  
idx = int(np.random.random()*15)
print("kick inserted on index", idx)        
od_b1[idx] = kick

nominal_B1_positions = np.repeat((np.arange(5)*params["nominal_step_size_B1"]),3)
true_B1_positions    = nominal_B1_positions * params["Nominal_LS_B1"] + np.cumsum(od_b1)
doros_B1_positions   = true_B1_positions / params["Doros_LS_B1"] + np.random.normal(size=true_B1_positions.shape) * params["Doros_noise_sigma"] 

print("true_B1_positions", true_B1_positions)

doros_B1_5positions = doros_B1_positions[1::3]
new_pos, od_std, rel_ls = avg_stepsize_corr_b1(doros_B1_5positions, nominal_B1_positions, 
                                                    sigma_multiplyer=3, outliers=1, 
                                                    outlier_removal_cutoff_sigma_ratio=1.0,
                                                    plot=True)

print("Measured relative ls:", rel_ls , "True relative ls:", params["Nominal_LS_B1"]/params["Doros_LS_B1"])


In [ ]:
# parameters
sigma_od = 4
kick     = 8

# code
bias = []
for i in range(10000):
    od_b1 = np.random.normal(size=15)*sigma_od  
    idx = int(np.random.random()*15)
    od_b1[idx] = kick

    nominal_B1_positions = np.repeat((np.arange(5)*params["nominal_step_size_B1"]),3)
    true_B1_positions    = nominal_B1_positions * params["Nominal_LS_B1"] + np.cumsum(od_b1)
    doros_B1_positions   = true_B1_positions / params["Doros_LS_B1"] + np.random.normal(size=true_B1_positions.shape) * params["Doros_noise_sigma"] 

    doros_B1_5positions = doros_B1_positions[1::3]
    new_pos, od_std, rel_ls = avg_stepsize_corr_b1(doros_B1_5positions, nominal_B1_positions, 
                        sigma_multiplyer=3, outliers=1, od_ls=False, plot=False, 
                        outlier_removal_cutoff_sigma_ratio=1.0)

    bias.append(rel_ls - params["Nominal_LS_B1"]/params["Doros_LS_B1"])

print("Bias of the resulting relative lengthscale:", np.mean(bias), ", std:", np.std(bias))